In [5]:
import time
import sqlite3
import requests
import datetime
import pandas as pd
import shioaji as sj
from shioaji import TickFOPv1, Exchange
from exchange_calendars import get_calendar
import mplfinance as mpf
import talib

from FuncBase import *

In [6]:
import os 
import shioaji as sj
from dotenv import load_dotenv

#永豐金API登入
load_dotenv()
api = sj.Shioaji(simulation=True)
api.login(os.getenv('YOUR_PERSON_ID'),os.getenv('YOUR_PASSWORD'),
    contracts_cb= lambda security_type: print(f"{repr(security_type)} fetch done"))

#連結DB Browser
connection = sqlite3.connect('work_data.db')

[14:10:35.392194] [12648] [pysolace/core/sol.cpp:499:SendRequest] [E] [thread 142929] Not ready


Exception ignored in: <function Shioaji.__del__ at 0x7f7ac0e68b80>
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/shioaji/shioaji.py", line 1211, in __del__
    self._solace.__del__()
  File "shioaji/backend/solace/api.pyx", line 2455, in shioaji.backend.solace.api.SolaceAPI.__del__
  File "shioaji/backend/solace/api.pyx", line 484, in shioaji.backend.solace.api.SolaceAPI.logout
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pysolace/__init__.py", line 275, in request
    raise TimeoutError(
TimeoutError: Topic: api/v1/auth/logout, Corr: c3, Client: PYAPI/F229985348/1010/052716/699000/101.12.179.26, payload: {'token': 'eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJuYmYiOjE3Mjg1MzgwMzcsImV4cCI6MTcyODYyNDQzNywic2ltdWxhdGlvbiI6dHJ1ZSwicGVyc29uX2lkIjoiRjIyOTk4NTM0OCIsInZlcnNpb24iOiIwLjUuMC5kZXYyIiwicDJwIjoiI1AyUC92OmlkY3NvbGFjZTAxL09yeWZrcXFtL1BZQVBJL0YyMjk5ODUzNDgvMTAxMC8wNTI3MTYvNjk

Response Code: 0 | Event Code: 0 | Info: host '203.66.91.161:80', hostname '203.66.91.161:80' IP 203.66.91.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up


<SecurityType.Index: 'IND'> fetch done
<SecurityType.Future: 'FUT'> fetch done


In [7]:
open_orders = {}
order_qty = {}
positions = {}

<SecurityType.Stock: 'STK'> fetch done
<SecurityType.Option: 'OPT'> fetch done


In [8]:
def place_cb(stat, msg):

    print('stat', stat)
    print('msg', msg)
    #'TFTDEAL' 下單成功
    if stat.value == 'TFTDEAL':

        code = msg['code']
        qty = int(msg['quantity'])
                
        order_qty[code] -= qty #扣除下單時委託張數

        if order_qty[code] == 0:
            open_orders = None

        position = positions.get(code,0)

        if msg['action'] == 'Sell':
            qty = qty * -1

        position += qty

        positions[code] = position
    
api.set_order_callback(place_cb)


Buy order

In [11]:
code = 'TXF202410'
codes = 'TXF'
current_price = 22782
qty = 1

contract = api.Contracts.Futures.get(codes)[code]

order = api.Order(
                  action=sj.constant.Action.Buy, 
                  price=current_price,
                  quantity=qty,
                  price_type="LMT",
                  order_type="ROD",
                  octype=sj.constant.FuturesOCType.Auto,
                  account=api.futopt_account)

trade = api.place_order(contract, order)

open_orders[code] = trade
order_qty[code] = qty

print(open_orders)
print(order_qty)
print(positions)

statResponse Code: 0 | Event Code: 0 | Info: host '218.32.76.102:80', hostname '218.32.76.102:80' IP 218.32.76.102:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up
 OrderState.FOrder
msg {'operation': {'op_type': 'New', 'op_code': '00', 'op_msg': ''}, 'order': {'id': '000003', 'seqno': '000003', 'ordno': '00002A', 'account': {'account_type': 'F', 'person_id': '', 'broker_id': 'F002000', 'account_id': '1553688', 'signed': False}, 'action': 'Buy', 'price': 22782, 'quantity': 1, 'order_type': 'ROD', 'price_type': 'LMT', 'oc_type': 'Auto', 'custom_field': ''}, 'status': {'id': '000003', 'exchange_ts': 1728886360.750907, 'order_quantity': 1, 'modified_price': 0, 'cancel_quantity': 0, 'web_id': 'Z'}, 'contract': {'security_type': 'FUT', 'exchange': 'TAIFEX', 'code': 'TXFJ4'}}
{'TXF202410': Trade(contract=Future(code='TXFJ4', symbol='TXF202410', name='臺股期貨10', category='TXF', delivery_month='202410', delivery_date='2024/10/16', underlying

Sell order

In [14]:
code = 'TXF202410'
codes = 'TXF'
current_price = 22782
position = positions.get(code, 0)

contract = api.Contracts.Futures.get(codes)[code]

order = api.Order(
                  action=sj.constant.Action.Sell, 
                  price=current_price,
                  quantity=positions,
                  price_type="LMT",
                  order_type="ROD",
                  octype=sj.constant.FuturesOCType.Auto,
                  account=api.futopt_account)

trade = api.place_order(contract, order)

open_orders[code] = trade
order_qty[code] = positions

print(open_orders)
print(order_qty)
print(positions)

{}

整合

In [ ]:
ticks = {}
kbars = {}
volume_today = {}

open_orders = {}
order_qty = {}
positions = {}

#設定callback 回傳資料型態
@api.on_tick_fop_v1()
def quote_callback(exchange: Exchange, tick: TickFOPv1):

    ticks = {}
    volume_today = {}

    ts = pd.to_datetime(tick.datetime)

    if tick.code[0].isalpha():
        code = tick.code[0:3]
    else:
        code = tick.code

    ts = pd.to_datetime(tick.datetime)

    if 5 < ts.hour < 9:
        return
    
    close = float(tick.close)
    volume = tick.volume
    vol_sum = tick.total_volume

    
    ticks[code].loc[ts] = [close, volume]
    volume_today[code] = vol_sum

def place_cb(stat, msg):

    if stat.value == 'TFTDEAL':

        code = msg['code']
        qty = int(msg['quantity'])
                
        order_qty[code] -= qty #扣除下單時委託張數

        if order_qty[code] == 0:
            open_orders = None

        position = positions.get(code,0)

        if msg['action'] == 'Sell':
            qty = qty * -1

        position += qty

        positions[code] = position
    
api.set_order_callback(place_cb)


In [ ]:
import time

def tbot(code):